In [ ]:
pip install pandas

In [ ]:
pip install bs4

In [9]:
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re
import json
import requests

In [14]:
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Instantiate options
options = Options()

# Add extra options
options.add_argument("--window-size=1920,1080")  # Set the window size
options.add_argument("--disable-infobars")  # Disable the infobars
options.add_argument("--disable-popup-blocking")  # Disable pop-ups
options.add_argument("--ignore-certificate-errors")  # Ignore certificate errors
options.add_argument('--headless') # ensure GUI is off
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Set path to chromedriver as per your configuration
homedir = "fabiodc"
options.binary_location = f"{homedir}/chrome-linux64/chrome"
webdriver_service = Service(f"{homedir}/chromedriver-linux64/chromedriver")

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=options)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [4]:

db_hash = {}

In [5]:
import requests

try:
    public_ip = requests.get('https://api.ipify.org').text
    print(f"Public IP Address: {public_ip}")
except requests.RequestException as e:
    print(f"Error retrieving public IP address: {e}")

Public IP Address: 34.106.249.2


In [8]:
url_first = 'https://www.amazon.it/deals?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
#page is a mutiple of 6 starting from 0, so page 1 is '' page 2 is '6' page 3 is '12' and so on
page = '6' #page1
url_last = '0%252C%2522presetId%2522%253A%2522deals-collection-lightning-deals%2522%252C%2522dealState%2522%253A%2522UPCOMING%2522%252C%2522dealType%2522%253A%2522LIGHTNING_DEAL%2522%252C%2522sorting%2522%253A%2522FEATURED%2522%257D'
url = url_first + page + url_last
#driver.get(url)
#html = driver.page_source
html = get_page_source(url)

In [11]:
pattern = r'\{"widgetId".*\}\)\n '
match = re.search(pattern, html)

In [12]:
deals_data = match.group().rstrip(")\n ")

In [13]:
# i'm getting less result for some reason both using selenium or requests, maybe is based on ip?
# it's confirmed that i get different offers based on ip, i need an italian ip
json.loads(deals_data)['prefetchedData']['aapiSearchDealsTotalCount']

167

In [ ]:
db_hash

In [ ]:
def parse_html_deals_page(html, db_hash):
  # Search for the pattern in the script content
  pattern = r'\{"widgetId".*\}\)\n '
  match = re.search(pattern, html)

  # fore some reason i'm not obtaining the first deal of the day
  deals_data = match.group().rstrip(")\n ")
  try:
    deals = json.loads(deals_data)['prefetchedData']['aapiGetDealsList'][0]['entities']
  except:
    print(html)
    return

  scraped_deals = {}
  for deal in deals:
    product_id=deal['entity']['details']['entity']['landingPage']['url'].split("/")[-1]
    # this is stupid, i need to improve this logic
    if product_id in db_hash:
      print("already_present")
      scraped_deals[product_id] = db_hash[product_id]
      continue
    start_time= deal['entity']['details']['entity']['startTime']['value']
    #start_time dosen't count early offers for prime members
    time_dic = deal['entity']['badge']['entity']['messaging']['content']['fragments'][1]
    promo_codes = get_promo_ids("https://www.amazon.it/dp/"+product_id)
    actual_start_time= time_dic['dateTime'] if 'dateTime' in time_dic else time_dic['countdownTimer']['targetTime']
    print(product_id + " : " + start_time + " : " + actual_start_time + " " + " ".join(promo_codes))
    scraped_deals[product_id] = {"start_time": start_time, "actual_start_time":actual_start_time, "promo_codes": promo_codes}

  return scraped_deals

In [ ]:
def get_promo_ids(url):
  res = requests.get(url)
  #to future me, i'm not stupid, i cannot use response.content.text because for some reason it remove some parts of the hmtl response, exactly the part that we need here
  html = res.content.decode(res.encoding)
  blocked_pattern = "api-services-support@amazon.com"
  if blocked_pattern in html:
    #blocked by amazon
    return get_promo_ids(url)
  promo_pattern = r'/promo/[^"]*'
  promos = re.findall(promo_pattern, html)
  return set([promo.split("/")[-1] for promo in promos])

In [ ]:
html = driver.page_source
parse_html_deals_page(html,db_hash)

In [7]:
def get_page_source(url):
  driver.get(url)
  return driver.page_source

In [ ]:
url_first = 'https://www.amazon.it/deals?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
#url_first = 'https://www.amazon.it/events/springdealdays?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
page = '' # first page
url_last = '0%252C%2522presetId%2522%253A%2522deals-collection-lightning-deals%2522%252C%2522dealState%2522%253A%2522UPCOMING%2522%252C%2522dealType%2522%253A%2522LIGHTNING_DEAL%2522%252C%2522sorting%2522%253A%2522FEATURED%2522%257D'
while True:
  url = url_first + page + url_last
  html = get_page_source(url)
  retry = 0
  # deals it's an hash where each key is a product_id
  deals = parse_html_deals_page(html,db_hash)
  if not deals:
    if (retry>2):
      break
    else:
      retry+=1
      # go on without incrementing page
      continue
  if deals:
    retry = 0
    for deal in deals:
      if deal not in db_hash:
        db_hash[deal] = deals[deal]
    page = str(int(page) + 1) if page.isdigit() else '1'
    print("page " + page)
  else:
    break

In [ ]:
len(db_hash)

181

In [ ]:
time_dic

{'dateTime': '2024-03-18T07:20:00.000Z'}

In [ ]:
db_array

In [ ]:
for el in db_array:
  promo_details_list = []
  for promo_id in el['promo_codes']:
    promo_details = get_promo_details(promo_id)
    promo_details_list.append(promo_details)
    #el['promo_details'] = promo_details
    print(promo_details)
  el['promo_details'] = promo_details_list

{'percent': '0', 'amount': '15.00'}
{'percent': '0', 'amount': '10.00'}
{'percent': '5', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '0', 'amount': '20.00'}
{'percent': '5', 'amount': ''}
{'percent': '10', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '0', 'amount': '50.00'}
{'percent': '10', 'amount': ''}
{'percent': '5', 'amount': ''}


In [ ]:
db_array

In [ ]:
!pip install selenium-wire

In [ ]:
driver.get("https://www.amazon.it/dp/B0CQCLF3HW")

In [ ]:
driver.get("https://www.amazon.it/dp/B0CQCLF3HW")
from seleniumwire.utils import decode

# Access requests via the `requests` attribute
response = driver.requests[0].response
body = decode(response.body, response.headers.get('Content-Encoding', 'identity'))
print(body)
for request in driver.requests:
    if request.response:
        #print(request.url)
        if (request.url == "https://www.amazon.it/dp/B0CQCLF3HW"):
          request.response

In [ ]:
test_product_url = "https://www.amazon.it/Dispenser-sapone-automatico-distributore-impermeabile/dp/B0C2ZBHYP5?pf_rd_r=ZBDJ72Q8EEP923K4QK63&pf_rd_t=Events&pf_rd_i=deals&pf_rd_p=625f4ee2-9c7d-4cb3-8da6-d5915b990e77&pf_rd_s=slot-14&ref=dlx_deals_gd_dcl_img_74_c80cb10e_dt_sl14_77&th=1"
test_product_url = "view-source:https://www.amazon.it/gp/product/B0C2V1113J"
driver.get(test_product_url)
driver.page_source

In [ ]:
print(driver.page_source)

In [ ]:
#questo metodo funzionva, ma a caso da selenium non si trova più l'url "/promo/", dal sorgente chrome c'è ancora
test_product_url = "https://www.amazon.it/dp/B0C2V1113J"
#driver.get(test_product_url)
#html = driver.page_source
#to future me, i'm not stupid, i cannot use response.content.text because for some reason it remove some parts of the hmtl response, exactly the part that we need here
while True
res = requests.get(test_product_url)
html = res.content.decode(res.encoding)
pattern = r'/promo/[^"]*'
promos = re.findall(pattern, html)
[promo.split("/")[-1] for promo in promos]

[]

In [ ]:
def get_promo_ids(url):
  res = requests.get(url)
  #to future me, i'm not stupid, i cannot use response.content.text because for some reason it remove some parts of the hmtl response, exactly the part that we need here
  html = res.content.decode(res.encoding)
  blocked_pattern = "api-services-support@amazon.com"
  if blocked_pattern in html:
    #blocked by amazon
    return get_promo_ids(url)
  promo_pattern = r'/promo/[^"]*'
  promos = re.findall(promo_pattern, html)
  return set([promo.split("/")[-1] for promo in promos])

In [ ]:
get_promo_ids(test_product_url)

blocked by amazon
blocked by amazon


['A132Y800WM9J03', 'A3O2Q4VBLX7WDT']

In [ ]:
res.content

b'<html>\n<head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Ci dispiace</title>\n</head>\n<body bgcolor="#FFFFFF" text="#000000">\n\n<!--\nPer discutere l\'accesso automatizzato ai dati di Amazon, contatta api-services-support@amazon.com.\nPer informazioni sulla migrazione alle nostre API, fare riferimento alle API di Marketplace all\'indirizzo https://developer.amazonservices.com/ref=rm_5_sv o alla nostra Product Advertising API all\'indirizzo https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac per i casi d\'uso pubblicitari.\n\nTo discuss automated access to Amazon data please contact api-services-support@amazon.com.\nFor information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.it/ref=rm_c_sv, or our Product Advertising API at https://programma-affiliazione.amazon.it/gp/a

In [ ]:
res.content.decode(res.encoding)

In [ ]:
import requests
res = requests.get("https://www.amazon.it/gp/product/B0C2V1113J")
res.content

In [ ]:
res.content.decode(res.encoding)

In [ ]:
driver.implicitly_wait(5)
driver.get("https://www.amazon.it/dp/B0C2V1113J")

html = driver.page_source
pattern = r'/promo/[^"]*'
promos = re.findall(pattern, html)
[promo.split("/")[-1] for promo in promos]

[]

In [ ]:
import requests
def get_promo_details(promo_id):
  base_promo_url = 'https://www.amazon.it/promotion/psp/'

  promo_url = base_promo_url + promo_id
  #promo_data = requests.get(promo_url).json()['cartProgressIndicatorResponse']['promotionTiers'][0]
  html = requests.get(promo_url).text
  pattern = r'let getCartItemsRequest = ({.*?});'
  search = re.search(pattern, html, re.DOTALL)
  if not search:
    return get_promo_details(promo_id)
  promo_data = search.group(1)

    # Try to convert the JSON string to a dictionary.
  promo_data = re.sub(r':\s*JSON\.stringify\(\w+\)', ': ""', promo_data)
  promo_dict = json.loads(promo_data)
  return {"percent": promo_dict['benefitPercent'].lower(), "amount": promo_dict['benefitAmount']}

get_promo_details('A3O2Q4VBLX7WDT')


{'percent': '5', 'amount': ''}

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime

soup = BeautifulSoup(html, 'html.parser')

# Find the parent div with the specified class prefix
parent_div = soup.select_one('div[class^="Grid-module__gridDisplayGrid"]')

# Find all child divs with the specified class prefix
divs = parent_div.select('div[class^="DealGridItem-module__dealItemDisplayGrid"]')

for div in divs:
    # Extract product URL
    product_url = div.select_one('a[class^="a-link-normal DealCard-module__linkOutlineOffset"]')['href']

    # Extract offer time and convert to datetime
    offer_time_str = div.select_one('time[class^="DealMessaging-module__dealMessaging"]').text.split(' ')[-1]
    offer_time = None
    try:
      #offer_time = datetime.strptime(offer_time_str, '%H:%M')
      offer_time = offer_time_str
    except:
      next

    print(f'Product URL: {product_url}')
    print(f'Offer Time: {offer_time}')
